In [30]:
from pymongo import MongoClient

client = MongoClient(
    "mongodb://mongo1:30001,mongo2:30002,mongo3:30003/?replicaSet=my-replica-set"
)

db = client.arxiv_db

# 1. Conteo total
total = db.articles.estimated_document_count()
print(f"📊 Total documentos: {total:,}")

# 2. Verificar consistencia entre nodos
for member in client.admin.command("replSetGetStatus")["members"]:
    print(f"🔹 Nodo {member['name']}: {'✅ HEALTHY' if member['health'] == 1 else '❌ PROBLEM'}")

# 3. Verificar muestras por categoría
print("\n📝 Muestra de documentos cs.AI:")
for doc in db.articles.find({"categories": "cs.AI"}, {"_id": 0, "title": 1}).limit(3):
    print(f" - {doc['title']}")

# 4. Verificar campos importantes
print("\n🔍 Verificación de campos críticos:")
stats = {
    "Con DOI": db.articles.count_documents({"doi": {"$exists": True}}),
    "Con PDF": db.articles.count_documents({"pdf_source": {"$exists": True}}),
    "Con autores": db.articles.count_documents({"authors": {"$exists": True, "$ne": []}})
}
# for k, v in stats.items():
#     print(f" - {k}: {v:,} ({v/total:.1%})")

📊 Total documentos: 2,161,943
🔹 Nodo mongo1:30001: ✅ HEALTHY
🔹 Nodo mongo2:30002: ✅ HEALTHY
🔹 Nodo mongo3:30003: ✅ HEALTHY

📝 Muestra de documentos cs.AI:
 - Calculating Valid Domains for BDD-Based Interactive Configuration
 - A study of structural properties on profiles HMMs
 - Bayesian approach to rough set

🔍 Verificación de campos críticos:


In [26]:
# db.drop_collection('articles')

{'nIndexesWas': 1,
 'ns': 'arxiv_db.articles',
 'ok': 1.0,
 '$clusterTime': {'clusterTime': Timestamp(1749605248, 1),
  'signature': {'hash': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
   'keyId': 0}},
 'operationTime': Timestamp(1749605248, 1)}

In [31]:
import json

def count_json_lines(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return sum(1 for line in f)

original_file = "arxiv-metadata-oai-snapshot.json"
total_original = count_json_lines(original_file)
print(f"Documentos en archivo original: {total_original}")

Documentos en archivo original: 2754926


In [32]:
archivos = [f"arxiv-part-{i:02d}.json" for i in range(11)]  # Ajusta el rango según tus archivos

total_particionado = 0
for archivo in archivos:
    count = count_json_lines(archivo)
    total_particionado += count
    print(f"{archivo}: {count} documentos")

print(f"\nTotal en archivos particionados: {total_particionado}")
print(f"Diferencia con original: {total_original - total_particionado}")

arxiv-part-00.json: 275492 documentos
arxiv-part-01.json: 275492 documentos
arxiv-part-02.json: 275492 documentos
arxiv-part-03.json: 275492 documentos
arxiv-part-04.json: 275492 documentos
arxiv-part-05.json: 275492 documentos
arxiv-part-06.json: 275492 documentos
arxiv-part-07.json: 275492 documentos
arxiv-part-08.json: 275492 documentos
arxiv-part-09.json: 275492 documentos
arxiv-part-10.json: 6 documentos

Total en archivos particionados: 2754926
Diferencia con original: 0


In [33]:
from pymongo import MongoClient

client = MongoClient("mongodb://mongo1:30001,mongo2:30002,mongo3:30003/?replicaSet=my-replica-set")
db = client["arxiv_db"]
collection = db["articles"]

# Conteo total
total_mongo = collection.count_documents({})
print(f"Documentos en MongoDB: {total_mongo}")

# Verificar por archivo (si agregaste un campo de origen)
if 'source_file' in collection.find_one({}, {'source_file': 1}):
    for archivo in archivos:
        count = collection.count_documents({"source_file": archivo})
        print(f"{archivo}: {count} documentos en MongoDB")

Documentos en MongoDB: 2754926
